In [0]:
# Mount drive
from google.colab import drive 
drive.mount("/content/drive")

In [0]:
# Locate and import files 
import sys
sys.path.append('/content/drive/COMP6248/Reproducibility/')
import synthetic
import seq2seq
import dilate_loss

In [0]:
# Install missing dependencies
!pip install tslearn

In [0]:
import numpy as np
import torch
from synthetic import create_synthetic_dataset, SyntheticDataset
from seq2seq import EncoderRNN, DecoderRNN, Net_GRU
from dilate_loss import dilate_loss
from torch.utils.data import DataLoader
import random
from tslearn.metrics import dtw, dtw_path
import matplotlib.pyplot as plt
import warnings
import warnings; warnings.simplefilter('ignore')

In [0]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
random.seed(0)

cuda:0


In [0]:
# parameters
batch_size = 100
N = 500
N_input = 84
N_output = 56  
sigma = 0.01
gamma = 0.01

In [0]:
import pandas as pd
from sklearn import preprocessing

train_data = pd.DataFrame(pd.read_csv('/content/drive/My Drive/Colab Notebooks/COMP6248/Reproducibility/ECG5000_TRAIN.tsv',delimiter='\t',header=None))
test_data = pd.DataFrame(pd.read_csv('/content/drive/My Drive/Colab Notebooks/COMP6248/Reproducibility/ECG5000_TEST.tsv',delimiter='\t',header=None))

In [0]:
train_data.head()

In [0]:
train_data = train_data.values
test_data = test_data.values
min_max = preprocessing.MinMaxScaler()
train_data = min_max.fit_transform(train_data)
test_data = min_max.fit_transform(test_data)
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

In [0]:
train_data.head()

In [0]:
# Load synthetic dataset
#X_train_input,X_train_target,X_test_input,X_test_target,train_bkp,test_bkp = create_synthetic_dataset(N,N_input,N_output,sigma)
X_train_input,X_train_target = train_data.loc[ : , 1:140-55], train_data.loc[ : , 140-55: ]
X_test_input,X_test_target = test_data.loc[ : , 1:140-55], test_data.loc[ : , 140-55: ] 
dataset_train = SyntheticDataset(X_train_input.to_numpy(),X_train_target.to_numpy(),np.zeros(500))
dataset_test  = SyntheticDataset(X_test_input.to_numpy(),X_test_target.to_numpy(),np.zeros(4500))
trainloader = DataLoader(dataset_train, batch_size=batch_size,shuffle=False, num_workers=1)
testloader  = DataLoader(dataset_test, batch_size=batch_size,shuffle=False, num_workers=1)  

In [0]:
X_train_input.head()

In [0]:
print(type(X_test_input))
print(X_test_input.shape,X_test_target.shape,len(trainloader))
for i,data in enumerate(trainloader, 0):
  inputs, target, _ = data

In [0]:
def train_model(net,loss_type, learning_rate, epochs=1000, gamma = 0.001,
                print_every=50,eval_every=50, verbose=1, Lambda=1, alpha=0.5):
    
    optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(epochs): 
        for i, data in enumerate(trainloader, 0):
            inputs, target, _ = data
            inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
            target = torch.tensor(target, dtype=torch.float32).to(device)
            batch_size, N_output = target.shape[0:2]                     

            # forward + backward + optimize
            outputs = net(inputs)
            loss_mse,loss_shape,loss_temporal = torch.tensor(0),torch.tensor(0),torch.tensor(0)
            
            if (loss_type=='mse'):
                loss_mse = criterion(target,outputs)
                loss = loss_mse                   
 
            if (loss_type=='dilate'):    
                loss, loss_shape, loss_temporal = dilate_loss(target,outputs,0.5, gamma, device)       

            if(loss_type=='soft_dtw'):
                loss,loss_shape,loss_temporal = dilate_loss(target,outputs,1, gamma, device)    

                  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()          
        
        if(verbose):
            if (epoch % print_every == 0):
                print('epoch ', epoch, ' loss ',loss.item(),' loss shape ',loss_shape.item(),' loss temporal ',loss_temporal.item())
                eval_model(net,testloader, gamma,verbose=1)

In [0]:
def eval_model(net,loader, gamma,verbose=1):   
    criterion = torch.nn.MSELoss()
    losses_mse = []
    losses_dtw = []
    losses_tdi = []   

    for i, data in enumerate(loader, 0):
        loss_mse, loss_dtw, loss_tdi = torch.tensor(0),torch.tensor(0),torch.tensor(0)
        # get the inputs
        inputs, target, breakpoints = data
        inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
        target = torch.tensor(target, dtype=torch.float32).to(device)
        batch_size, N_output = target.shape[0:2]
        outputs = net(inputs)
         
        # MSE    
        loss_mse = criterion(target,outputs)    
        loss_dtw, loss_tdi = 0,0
        # DTW and TDI
        for k in range(batch_size):         
            target_k_cpu = target[k,:,0:1].view(-1).detach().cpu().numpy()
            output_k_cpu = outputs[k,:,0:1].view(-1).detach().cpu().numpy()

            loss_dtw += dtw(target_k_cpu,output_k_cpu)
            path, sim = dtw_path(target_k_cpu, output_k_cpu)   
                       
            Dist = 0
            for i,j in path:
                    Dist += (i-j)*(i-j)
            loss_tdi += Dist / (N_output*N_output)            
                        
        loss_dtw = loss_dtw /batch_size
        loss_tdi = loss_tdi / batch_size

        # print statistics
        losses_mse.append( loss_mse.item() )
        losses_dtw.append( loss_dtw )
        losses_tdi.append( loss_tdi )

    print( ' Eval mse= ', np.array(losses_mse).mean() ,' dtw= ',np.array(losses_dtw).mean() ,' tdi= ', np.array(losses_tdi).mean()) 

In [0]:

encoder = EncoderRNN(input_size=1, hidden_size=128, num_grulstm_layers=1, batch_size=batch_size).to(device)
decoder = DecoderRNN(input_size=1, hidden_size=128, num_grulstm_layers=1,fc_units=16, output_size=1).to(device)
net_gru_dilate = Net_GRU(encoder,decoder, N_output, device).to(device)
train_model(net_gru_dilate,loss_type='dilate',learning_rate=0.001, epochs=1000, gamma=gamma, print_every=50, eval_every=50,verbose=1)

encoder = EncoderRNN(input_size=1, hidden_size=128, num_grulstm_layers=1, batch_size=batch_size).to(device)
decoder = DecoderRNN(input_size=1, hidden_size=128, num_grulstm_layers=1,fc_units=16, output_size=1).to(device)
net_gru_mse = Net_GRU(encoder,decoder, N_output, device).to(device)
train_model(net_gru_mse,loss_type='mse',learning_rate=0.001, epochs=1000, gamma=gamma, print_every=50, eval_every=50,verbose=1)

encoder = EncoderRNN(input_size=1, hidden_size=128, num_grulstm_layers=1, batch_size=batch_size).to(device)
decoder = DecoderRNN(input_size=1, hidden_size=128, num_grulstm_layers=1,fc_units=16, output_size=1).to(device)
net_gru_dtw = Net_GRU(encoder,decoder, N_output, device).to(device)
train_model(net_gru_dtw,loss_type='soft_dtw',learning_rate=0.001, epochs=1000, gamma=gamma, print_every=50, eval_every=50,verbose=1)

In [0]:
# Visualize results
gen_test = iter(testloader)
test_inputs, test_targets, breaks = next(gen_test)

test_inputs  = torch.tensor(test_inputs, dtype=torch.float32).to(device)
test_targets = torch.tensor(test_targets, dtype=torch.float32).to(device)
criterion = torch.nn.MSELoss()

nets = [net_gru_mse,net_gru_dilate,net_gru_dtw]

for ind in range(1,51):
    plt.figure()
    plt.rcParams['figure.figsize'] = (17.0,5.0)  
    k = 1
    for net in nets:
        pred = net(test_inputs).to(device)

        input = test_inputs.detach().cpu().numpy()[ind,:,:] 
        target = test_targets.detach().cpu().numpy()[ind,:,:]
        preds = pred.detach().cpu().numpy()[ind,:,:]

        plt.subplot(1,3,k)
        plt.plot(range(0,N_input +1) ,input,label='input',linewidth=3)
        plt.plot(range(N_input-1,N_input+N_output), np.concatenate([ input[N_input-1:N_input], target ]) ,label='target',linewidth=3)   
        plt.plot(range(N_input-1,N_input+N_output),  np.concatenate([ input[N_input-1:N_input], preds ])  ,label='prediction',linewidth=3)       
        #plt.xticks(range(0,40))
        plt.legend()
        k = k+1

    plt.show()